In [1]:
%reload_ext autotime
import pandas as pd
import plotly.io as pio
import plotly.express as px # Plotting
from tqdm import tqdm
tqdm.pandas()
import statsmodels.formula.api as smf
from sklearn.ensemble import RandomForestRegressor
pio.renderers.default = 'notebook'
pd.options.plotting.backend = "plotly"
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 100)

In [31]:
df = pd.read_excel("LLM_results.xlsx")
df = df[(df.timeLeft == "closed") & (df.goal > 0)]
df.sentiment = df.sentiment.astype(str).str.split(" ").str[0].astype(float)
df.truth = df.truth.astype(str).str.split(" ").str[0].astype(float)
df.urgency = df.urgency.astype(str).str.split(" ").str[0].astype(float)
df["Target_completion"] = df.amountRaised / df.goal * 100
df

,Index,title,pitch,hero,uri,amountRaised,goal,timeLeft,action,actionUri,start,end,n_questions,location,n_donors,description,use_of_funds,whos_involved,moderated,n_updates,updates,_21,text,condition,ICD10,ICD,name,gender,age,age_group,ethnicity,urgency,sentiment,truth,notes,smiling,deservingness,attractiveness,use,Target_completion
30,31,Helping a person in need,Helping out a person in need that would drop his hat to help someone else,https://static.givealittle.co.nz/assets/hero/ed760e80-7e10-4078-b7a8-b2ec0104419b-320:False,https://givealittle.co.nz/cause/helping-a-person-in-need,1700.0,2000.0,closed,Read more,https://givealittle.co.nz/cause/helping-a-person-in-need,29 May 2025,8 Jun 2025,0,"Blenheim, Marlborough",54.0,"Tok had a serious injury while running — he tore a muscle, which led to internal bleeding and ev...",medical care where needed and general stuff around the house if needed also and utility's,"Created by, and paying to a verified bank account of,\r\nMatthew bergman on behalf of Tok H...",Page Moderated,3.0,Big thanks\n7 June 2025\nHey team huge thanks to all that have donated Tok is humbled beyond wor...,NaN,Helping a person in need Helping a person in need Helping out a person in need that would drop h...,Muscle tear leading to internal bleeding and infection,S86.0,"Injury, poisoning and certain other consequences of external causes",Tok,Male,35,15-64,Pacific Islander,90.0,70.0,95.0,Age estimated based on appearance; ethnicity inferred from image; sentiment reflects a mix of gr...,1.0,95,60,medical expenses,85.000000
55,56,Please help Walter 🙏 to get the veterinary care he needs.,Please help us 🙏 Walty needs bloodwork to rule out cancer 😪,https://static.givealittle.co.nz/assets/hero/a5763829-1835-4780-adc3-b2e500bd5557-320:False,https://givealittle.co.nz/cause/please-help-walter-to-get-the-veterinary-care-he,380.0,500.0,closed,Read more,https://givealittle.co.nz/cause/please-help-walter-to-get-the-veterinary-care-he,22 May 2025,1 Jun 2025,0,Otago,14.0,"Walter has been my life for 15 years, since he was a pup. It has always being just the two of us...","The money raised will go into Walts vet costsFull bloods, medication etc at the moment is quoted...","Created by, and paying to a verified bank account of,\r\nKerri Back",Page Moderated,1.0,Bloodwork done.\n23 May 2025\nThank you so much for your kind donations.\nWalty has had his bloo...,NaN,Please help Walter 🙏 to get the veterinary care he needs. Please help Walter 🙏 to get the veteri...,Pancreatitis,K85,11,Walter,Male,15,15-64,Unknown (animal),90.0,70.0,95.0,"The primary subject is an animal named Walter, not a human. The text mentions pancreatitis as th...",1.0,90,80,medical expenses,76.000000
58,59,Medical Treatment for Atish Lal for Coronary Artery Disease,medical Treatment,https://static.givealittle.co.nz/assets/hero/0a81d0ab-2bfc-4890-8049-b2e3017fea04-320:False,https://givealittle.co.nz/cause/medical-treatment-for-atish-lal-for-coronary,380.0,14900.0,closed,Read more,https://givealittle.co.nz/cause/medical-treatment-for-atish-lal-for-coronary,20 May 2025,6 Jun 2025,0,Auckland,6.0,"Dear Family and Friends,\nI hope this message finds you well.\nI am reaching out today with a hu...","Estimated costs for Treatment & Travel:- PTCA treatment package + home stay : $11,000 USD- Reser...",Created by\r\nShaneel Prasad \n\n\n\n\r\n Paying to a verified bank account of\r\nPriya Go...,Page Moderated,NaN,NaN,NaN,Medical Treatment for Atish Lal for Coronary Artery Disease Medical Treatment for Atish Lal for ...,Coronary Artery Disease,I25,I - Diseases of the circulatory system,Atish Lal,Male,35,15-64,Indian,90.0,70.0,95.0,Age estimated based on typical age range for coronary artery disease; ethnicity inferred from th...,0.0,85,50,medical expenses,2.550336
73,74,Help me to reclaim my freedom.,Help my to reclaim my freedom and contribute to my community.,https://static.givealittle.co.nz/assets/hero/0e91c689-ef0e-4c5c-aa9d-b2e0010f721b-320:False,https://givealittle.co.nz

In [32]:
df.gender.value_counts().head(10)

gender
Female              1579
Male                1452
female               679
Not specified        175
male                 161
Transgender           33
Transgender Male      14
Unknown (animal)      14
Male and Female       13
Unknown               13
Name: count, dtype: int64

In [33]:
df.gender = df.gender.str.lower().where(df.gender.str.lower().isin(['male', 'female']), 'other')
df.gender.value_counts()

gender
female    2258
male      1613
other      540
Name: count, dtype: int64

In [34]:
df.ethnicity.value_counts().head(20)

ethnicity
Caucasian                              960
Caucasian (based on image)             623
Caucasian (assumed based on image)     552
South Asian                            171
Māori                                  109
Asian                                  105
European/Caucasian                      80
Pacific Islander                        70
Caucasian (inferred from image)         61
European/Caucasian (based on image)     54
African                                 50
Pacific Islander (based on image)       46
Unknown (animal)                        46
Not specified (image not provided)      42
Filipino                                42
Indian                                  41
Caucasian (inferred from the image)     38
European/White (based on image)         36
South Asian/Indian                      33
East Asian                              32
Name: count, dtype: int64

In [35]:
df.ethnicity = df.ethnicity.str.extract(r"(.+?)(?= \(|$)")
df.ethnicity.value_counts().head(20)

ethnicity
Caucasian                  2284
South Asian                 214
Māori                       209
Asian                       161
European/Caucasian          148
Pacific Islander            144
Not specified               121
Unknown                     121
European/White               76
Maori                        63
African                      57
Filipino                     54
Indian                       49
East Asian                   42
White                        39
Mixed                        38
South Asian/Indian           35
Filipino/Asian               33
European/New Zealand         33
Latin American/Hispanic      29
Name: count, dtype: int64

In [36]:
df.ethnicity = df.ethnicity.replace({
    "European/Caucasian": "Caucasian",
    "European/White": "Caucasian",
    "European/White": "Caucasian",
    "European/New Zealand": "Caucasian",
    "Maori": "Māori"
})
top_20 = df.ethnicity.value_counts().head(20)
display(top_20)
df.ethnicity = df.ethnicity.where(df.ethnicity.isin(top_20.index), "other")

ethnicity
Caucasian                  2541
Māori                       272
South Asian                 214
Asian                       161
Pacific Islander            144
Unknown                     121
Not specified               121
African                      57
Filipino                     54
Indian                       49
East Asian                   42
White                        39
Mixed                        38
South Asian/Indian           35
Filipino/Asian               33
Latin American/Hispanic      29
Fijian                       25
Hispanic/Latino              22
Not applicable               18
European/New Zealander       16
Name: count, dtype: int64

In [37]:
df.condition.str.lower().value_counts().head(20)

condition
cancer                     192
gender dysphoria           135
breast cancer              135
stroke                      75
multiple sclerosis          65
endometriosis               45
bowel cancer                44
type 1 diabetes             39
melanoma                    38
obesity                     37
none specified              36
cerebral palsy              35
terminal cancer             33
infertility                 32
mental health issues        31
covid-19                    27
lung cancer                 27
coronary artery disease     27
hearing loss                26
brain tumor                 26
Name: count, dtype: int64

In [38]:
df.sentiment.value_counts()

sentiment
70.0     1717
80.0      603
50.0      524
60.0      493
90.0      422
30.0      258
85.0      235
20.0       64
95.0       35
40.0       28
75.0       26
10.0        5
100.0       1
Name: count, dtype: int64

In [39]:
smf.ols(formula="Target_completion ~ sentiment + smiling + gender + truth", data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Target_completion   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.5780
Date:                Mon, 23 Jun 2025   Prob (F-statistic):              0.717
Time:                        12:11:59   Log-Likelihood:                -35588.
No. Observations:                4411   AIC:                         7.119e+04
Df Residuals:                    4405   BIC:                         7.123e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          75.3435    266.765      0.282      0.778    -447.650     598.337
gender[T.male]     41.9276     25.599      1.638      0.102      -8.259      92.114
gender[T.other]    12.6667     38.188      0.332      0.740     -62.200      87.534
sentiment           0.3024      0.808      0.374      0.708      -1.281       1.886
smiling            12.0683     24.375      0.495      0.621     -35.720      59.857
truth              -0.5875      3.071     -0.191      0.848      -6.608       5.433
==============================================================================
Omnibus:                    13559.696   Durbin-Watson:                   2.004
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       1099445460.571
Skew:                          46.137   Prob(JB):                         0.00
Kurtosis:                    2447.076   Cond. No.                     2.62e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.62e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [40]:
smf.ols(formula="Target_completion ~ sentiment + smiling + gender + truth + urgency + ethnicity", data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Target_completion   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.3153
Date:                Mon, 23 Jun 2025   Prob (F-statistic):               1.00
Time:                        12:12:10   Log-Likelihood:                -35586.
No. Observations:                4411   AIC:                         7.123e+04
Df Residuals:                    4384   BIC:                         7.140e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               18.7330    292.516      0.064      0.949    -554.747     592.213
gender[T.male]                          43.9074     25.822      1.700      0.089      -6.716      94.531
gender[T.other]                         28.5860     42.725      0.669      0.503     -55.177     112.349
ethnicity[T.Asian]                       0.6342    119.793      0.005      0.996    -234.221     235.490
ethnicity[T.Caucasian]                  50.6678    104.354      0.486      0.627    -153.918     255.254
ethnicity[T.East Asian]                  1.5373    157.629      0.010      0.992    -307.495     310.569
ethnicity[T.European/New Zealander]     10.9025    219.228      0.050      0.960    -418.896     440.701
ethnicity[T.Fijian]                      4.2426    185.896      0.023      0.982    -360.207     368.692
ethnicity[T.Filipino]                  -17.9307    147.171     -0.122      0.903    -306.461     270.600
ethnicity[T.Filipino/Asian]            -31.6723    169.751     -0.187      0.852    -364.470     301.125
ethnicity[T.Hispanic/Latino]            13.9629    195.204      0.072      0.943    -368.735     396.661
ethnicity[T.Indian]                    -19.0436    151.388     -0.126      0.900    -315.841     277.754
ethnicity[T.Latin American/Hispanic]    -2.3655    176.925     -0.013      0.989    -349.228     344.497
ethnicity[T.Mixed]                      12.5670    162.744      0.077      0.938    -306.494     331.628
ethnicity[T.Māori]                      12.3746    113.076      0.109      0.913    -209.311     234.061
ethnicity[T.Not applicable]             21.9954    211.458      0.104      0.917    -392.569     436.560
ethnicity[T.Not specified]              10.5634    127.976      0.083      0.934    -240.334     261.461
ethnicity[T.Pacific Islander]            3.7364    121.340      0.031      0.975    -234.151     241.623
ethnicity[T.South Asian]               -19.3334    115.996     -0.167      0.868    -246.745     208.078
ethnicity[T.South Asian/Indian]        -25.1335    166.575     -0.151      0.880    -351.705     301.438
ethnicity[T.Unknown]                     6.7182    125.145      0.054      0.957    -238.628     252.065
ethnicity[T.White]                       3.0813    161.579      0.019      0.985    -313.695     319.858
ethnicity[T.other]                      39.3088    110.101      0.357      0.721    -176.544     255.162
sentiment                                0.7953      0.868      0.916      0.360      -0.907       2.498
smiling                                  1.6201     25.800      0.063      0.950     -48.961      52.201
truth                                   -2.2542      3.265     -0.690      0.490      -8.655       4.146
urgency                               

In [41]:
model = RandomForestRegressor()
cols = "sentiment + smiling + gender + truth + urgency + ethnicity".split(" + ")
X = pd.get_dummies(df[cols])
y = df["Target_completion"]
model.fit(X, y)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [42]:
list(zip(cols, model.feature_importances_))

[('sentiment', np.float64(0.39299137046674376)),
 ('smiling', np.float64(0.11223198470201788)),
 ('gender', np.float64(0.0742451045880252)),
 ('truth', np.float64(0.19745166547225454)),
 ('urgency', np.float64(0.02427764589988352)),
 ('ethnicity', np.float64(0.03648669228377975))]